In [5]:
import os
from dotenv import load_dotenv

load_dotenv()
uri = os.getenv('uri')
user = os.getenv('user')
password = os.getenv('password')

In [1]:
## Tech role ingestion class
from neo4j import GraphDatabase

class Neo4jGraph:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_graph(self):
        with self.driver.session() as session:
            session.write_transaction(self._create_nodes)
            session.write_transaction(self._create_relationships)

    @staticmethod
    def _create_nodes(tx):
        domains = ['Frontend', 'Backend']
        for domain in domains:
            tx.run("MERGE (d:Domain {name: $name})", name=domain)
        
        languages = ['Python', 'JavaScript', 'Ruby', 'Java', 'TypeScript', 'PHP',  'Node.js', 'C#']
        for language in languages:
            tx.run("MERGE (l:Language {name: $name})", name=language)
        
        frameworks = [
            {'name': 'React', 'domain': 'Frontend'},
            {'name': 'Angular', 'domain': 'Frontend'},
            {'name': 'Vue.js', 'domain': 'Frontend'},
            {'name': 'Svelte', 'domain': 'Frontend'},
            {'name': 'Ember.js', 'domain': 'Frontend'},
            {'name': 'Backbone.js', 'domain': 'Frontend'},
            {'name': 'Django', 'domain': 'Backend'},
            {'name': 'Flask', 'domain': 'Backend'},
            {'name': 'Ruby on Rails', 'domain': 'Backend'},
            {'name': 'Spring Boot', 'domain': 'Backend'},
            {'name': 'Laravel', 'domain': 'Backend'},
            {'name': 'Express.js', 'domain': 'Backend'},
            {'name': 'NestJS', 'domain': 'Backend'},
            {'name': 'Phoenix', 'domain': 'Backend'},
            {'name': 'Gin', 'domain': 'Backend'},
            {'name': 'ASP.NET Core', 'domain': 'Backend'},
            {'name': 'Symfony', 'domain': 'Backend'},
            {'name': 'Play Framework', 'domain': 'Backend'}
        ]
        for framework in frameworks:
            tx.run("MERGE (f:Framework {name: $name})", name=framework['name'])
            
    @staticmethod
    def _create_relationships(tx):
        framework_domain_relations = [
            {'framework': 'React', 'domain': 'Frontend'},
            {'framework': 'Angular', 'domain': 'Frontend'},
            {'framework': 'Vue.js', 'domain': 'Frontend'},
            {'framework': 'Svelte', 'domain': 'Frontend'},
            {'framework': 'Ember.js', 'domain': 'Frontend'},
            {'framework': 'Backbone.js', 'domain': 'Frontend'},
            {'framework': 'Django', 'domain': 'Backend'},
            {'framework': 'Flask', 'domain': 'Backend'},
            {'framework': 'Ruby on Rails', 'domain': 'Backend'},
            {'framework': 'Spring Boot', 'domain': 'Backend'},
            {'framework': 'Laravel', 'domain': 'Backend'},
            {'framework': 'Express.js', 'domain': 'Backend'},
            {'framework': 'NestJS', 'domain': 'Backend'},
            {'framework': 'Phoenix', 'domain': 'Backend'},
            {'framework': 'Gin', 'domain': 'Backend'},
            {'framework': 'ASP.NET Core', 'domain': 'Backend'},
            {'framework': 'Symfony', 'domain': 'Backend'},
            {'framework': 'Play Framework', 'domain': 'Backend'}
        ]
        for relation in framework_domain_relations:
            tx.run("""
                MATCH (f:Framework {name: $framework_name})
                MATCH (d:Domain {name: $domain_name})
                MERGE (f)-[:IN_DOMAIN]->(d)
            """, framework_name=relation['framework'], domain_name=relation['domain'])
        
        framework_language_relations = [
            {'framework': 'React', 'languages': ['JavaScript']},
            {'framework': 'Angular', 'languages': ['TypeScript', 'JavaScript']},
            {'framework': 'Vue.js', 'languages': ['JavaScript']},
            {'framework': 'Svelte', 'languages': ['JavaScript']},
            {'framework': 'Ember.js', 'languages': ['JavaScript']},
            {'framework': 'Backbone.js', 'languages': ['JavaScript']},
            {'framework': 'Django', 'languages': ['Python']},
            {'framework': 'Flask', 'languages': ['Python']},
            {'framework': 'Ruby on Rails', 'languages': ['Ruby']},
            {'framework': 'Spring Boot', 'languages': ['Java']},
            {'framework': 'Laravel', 'languages': ['PHP']},
            {'framework': 'Express.js', 'languages': ['JavaScript', 'Node.js']},
            {'framework': 'NestJS', 'languages': ['TypeScript', 'Node.js']},
            {'framework': 'Phoenix', 'languages': ['Elixir']},
            {'framework': 'Gin', 'languages': ['Go']},
            {'framework': 'ASP.NET Core', 'languages': ['C#']},
            {'framework': 'Symfony', 'languages': ['PHP']},
            {'framework': 'Play Framework', 'languages': ['Scala', 'Java']}
        ]
        for relation in framework_language_relations:
            for lang in relation['languages']:
                tx.run("""
                    MATCH (f:Framework {name: $framework_name})
                    MATCH (l:Language {name: $lang_name})
                    MERGE (f)-[:REQUIRES]->(l)
                """, framework_name=relation['framework'], lang_name=lang)
                
        framework_dependencies = [
            {'framework': 'NestJS', 'requires': 'Express.js'},
            {'framework': 'Angular', 'requires': 'TypeScript'},
            {'framework': 'React', 'requires': 'JavaScript'}
        ]
        
        for dependency in framework_dependencies:
            tx.run("""
                MATCH (f1:Framework {name: $framework_name})
                MATCH (f2:Framework {name: $required_framework})
                MERGE (f1)-[:REQUIRES]->(f2)
            """, framework_name=dependency['framework'], required_framework=dependency['requires'])
            
        ## Ingesting Related_to skills
        tx.run("""MATCH (a:Framework{name: 'Flask'}), (b:Framework{name: 'Django'})
                MERGE (a)-[:RELATED_TO]->(b)
                RETURN a,b""")
        
        tx.run("""MATCH (a:Framework{name: 'Play Framework'}), (b:Framework{name: 'Spring Boot'})
                MERGE (a)-[:RELATED_TO]->(b)
                RETURN a,b""")

if __name__ == "__main__": 
    
    graph = Neo4jGraph(uri, user, password)
    graph.create_graph()
    graph.close()


/tmp/ipykernel_3626882/3692834683.py:12: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_nodes)
/tmp/ipykernel_3626882/3692834683.py:13: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_relationships)
